<h1><b>How might we decrease the probability of death for those in disaster areas through a national search and rescue application pared with wrist-mounted wearable technology for search and rescue teams?</b></h1>

<h3><b>Assumption: </b>Search and rescue teams with access to more up-to-date information regarding the location of survivors would be more able to find and rescue survivors.</h3>

<h3><b>Question: </b>Would search and rescue operations benefit from more survivor information?</h3>

<h3><b>Hypothesis: </b>Search and rescue teams with access to the locational data of survivors would be able to find a greater number of survivors in the 48 hours following a natural disaster.</h3>

<h3><b>Setup: </b></h3>
<h5><ul>
    <li>x -> locations</li>
    <li>n -> survivors</li>
    <li>p -> probability of people in house</li>
    <li>d_max -> max distance a searcher can move during a discrete time interval</li>
    <li>T -> Total number of timesteps in the simulation</li>
    <li>t -> current timestep</li>
    <li>dt -> how often agent_2 receives the location of a cluster of survivors</li>
</ul></h5>

<h3><b>Experiment: </b></h3>
<h5><p>
Two agents, agent_1 and agent_2 are given a set amount of time T to traverse through a set of x discrete locations to find the largest proportion of survivors.  Agent_1 will always search the current location for survivors before progressing to the next location (x[i+1]) and searching that location.  Agent_2 will search in the linear method agent_1 exhibits, but when it receives information from base campe evert dt timesteps, agent_2 will set it's next location as the location from base camp.  For agent_2, there is a maximum distance agent_2 can move per timestep (d_max).  Therefore, if agent_2 is given a new location that is farther than the maximum distance from it's current location, agent_2 will take multiple timesteps to get there.  After T timesteps, the number of "survivors" found by each agent will be compared.

As time progresses, p, T, and dt will be varied to determine the effectiveness of agents given different parameters.
</p></h5>

<h3><b>Conclusion: </b></h3>

<h3><b>Next Setps: </b></h3>

All data is stored in "../louisiana.csv"

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
louisiana_locations = pd.read_csv('./louisiana_data.csv')
fire_stations = pd.read_csv('./stations.csv', delimiter=';')
louisiana_population = 4649000
TOTAL_POPULATION = louisiana_locations / 2
DISCOVERY_PROBABILITY = 0.5
# assuming the search and rescue teams move at about 15 miles per hour
# 15 miles per hour * 1609 meters/mile * 1 hour/3600 seconds
MOVE_SPEED = 15 * 1609 /3600
MAX_PEOPLE_PER_LOCATION = 10
TIME_TO_RESCUE_PERSON = 3 * 60
SAVE_CUTOFF_DISTANCE = 1
INFORM_DISTANCE_CUTOFF = 10 * 1609
TIME_BETWEEN_SENDS = 10 * 60

In [ ]:
class location():
    """
    Singular location for the search and rescue team to search.
    """
    def __init__(self, x_coordinate: float, y_coordinate: float):
        self.x = x_coordinate
        self.y = y_coordinate
        self.searched = False
        self.position = np.zeros(1, 2)
        self.position[0] = self.x
        self.position[1] = self.y
        self.num_people = 0

    def populate_with_survivors(self, num_people):
        self.num_people = num_people

    def __str__(self):
        outstring = "{["
        outstring += self.x + ';' + self.y + ';' + self.searched + ';' + self.num_people + "]|"
        return outstring

    def create_from_file(self, string):
        information = string [1:-2]
        information.split(';')
        self.x = float(information[0])
        self.y = float(information[1])
        self.searched = bool(information[2])
        self.position = np.zeros(1, 2)
        self.position[0] = self.x
        self.position[1] = self.y
        self.num_people = int(information[3])

In [ ]:
locations = []
num_survivors = TOTAL_POPULATION
for index, series in louisiana_locations.iterrows():
    x = series['x']
    y = series['y']
    rand_num = np.random.randint(0, 2)
    survivors_at_location = 0
    if rand_num == 1:
        survivors_at_location = np.random.randint(0, MAX_PEOPLE_PER_LOCATION + 1)
    new_location = location(x, y)
    new_location.populate_with_survivors(survivors_at_location)
    locations.append(new_location)

In [ ]:
locations_with_survivors = []
for loc in locations:
    if loc.num_people != 0:
        locations_with_survivors.append(loc)

Populate the locations given in "louisiana_data.csv" with a random number of people [0, MAX_PEOPLE_PER_LOCATION] at a probabiliyt of 0.5

In [ ]:
class FireStation(location):
    def __init__(self, x_location: float, y_location: float, num_search_teams: int):
        super(x_location, y_location)
        self.num_teams = num_search_teams

In [ ]:
stations = []
for index, series in fire_stations.iterrows():
    x = series['x']
    y = series['y']
    new_station = FireStation(x, y, 1)
    stations.append(new_station)

Make station locations based on the fire station locations extracted from the internet.

In [ ]:
class Agent():
    """
    Basic agent class all other agents will extend from
    """
    def __init__(self, start_location: FireStation):
        self.start_position = start_location.position
        self.current_position = self.start_location.position
        self.num_saved = 0
        self.target_location = self.plan_next_destination()
        self.cooldown = 0
        self.checked_locations = []
        self.survivor_locations = []
        self.unique_name = ""

    def tick(self, locations: list[location], people_location: location):
        if self.cooldown >= 0:
            self.cooldown -= 1
            return
        elif self.target_location is None:
            self.plan_next_destination(locations)
        
        if np.linalg.norm(self.target_location - self.current_position) < SAVE_CUTOFF_DISTANCE:
            self.save_survivors
        else:
            move_direction = (self.target_location.position - self.current_position) / np.linalg.norm(self.target_location.position - self.current_position)
            self.current_position = move_direction * MOVE_SPEED

    def plan_next_destination(self, locations: list[location]):
        pass

    def save_survivors(self):
        pass

    def save_state(self, outfile: str):
        file = open(outfile, 'a')
        outstring = ""
        outstring += str(self.start_position[0]) + ',' + str(self.start_position[1]) + ','
        outstring += str(self.current_position[0]) + ',' + str(self.current_position[1]) + ','
        outstring += str(self.num_saved) + ','
        outstring += str(self.target_location) + '},'
        outstring += str(self.cooldown) + ','

    def load_state(
        self, start_x, start_y, current_x, current_y, num_saved, target_location, cooldown, checked_locations,
        survivor_locations, unique_name
    ):
        self.start_position = np.zeros(1, 2)
        self.start_position[0] = start_x
        self.start_position[1] = start_y
        self.current_position = np.zeros(1, 2)
        self.current_position[0] = current_x
        self.current_position[1] = current_y
        self.num_saved = num_saved
        self.target_location = target_location
        self.cooldown = cooldown
        self.checked_locations = checked_locations
        self.checked_locations
        self.survivor_locations = survivor_locations
        self.unique_name = unique_name

In [ ]:
class h2h_agent(Agent):
    """
    The current agent that will move from location to location searching each subsequent non-searched location for
    survivors
    """
    def __init__(self, start_location: FireStation):
        super(start_location)
        self.unique_name = "h2h_agent"

    def tick(self, locations: list[location], people_location: location):
        super().tick(locations)

    def plan_next_destination(self, locations: list[location]):
        closest_distance = np.linalg.norm(locations[0].position - self.current_position)
        closest_location = locations[0]
        for i in range(0, len(locations)):
            if locations[i] not in self.checked_locations:
                distance = np.linalg.norm(locations[i].position - self.current_position)
                if distance < closest_distance:
                    closest_distance = distance
                    closest_location = locations[i]
        self.target_location = closest_location

    def save_survivors(self):
        self.cooldown = self.target_location.num_people * TIME_TO_RESCUE_PERSON
        self.checked_locations.append(self.target_location)
        self.target_location = None


In [ ]:
class informed_agent_1(Agent):
    """
    A semi-informed agent that recieves the location of a group of survivors every 10 minutes
    (assuming it is the closest agent to the survivor, otherwise it conducts house to house searches)
    """
    def __init__(self, start_location: FireStation):
        super(start_location)
        self.unique_name = "informed_agent_1"

    def tick(self, locations: list[location], people_location: location):
        if people_location is not None:
            self.survivor_locations.append(people_location)
        super().tick(locations)

    def plan_next_destination(self, locations: list[location]):
        if len(self.survivor_locations) == 0:
            closest_distance = np.linalg.norm(locations[0].position - self.current_position)
            closest_location = locations[0]
            for i in range(0, len(locations)):
                if not locations[i].searched:
                    distance = np.linalg.norm(locations[i].position - self.current_position)
                    if distance < closest_distance:
                        closest_distance = distance
                        closest_location = locations[i]
            self.target_location = closest_location
        else:
            closest_distance = np.linalg.norm(self.survivor_locations[0].position - self.current_position)
            closest_location = self.survivor_locations[0]
            for i in range(0, len(self.survivor_locations)):
                distance = np.linalg.norm(self.survivor_locations[i].position - self.current_position)
                if distance < closest_distance:
                    closest_distance = distance
                    closest_location = self.survivor_locations[i]
            self.target_location = closest_location

    def save_survivors(self):
        self.cooldown = self.target_location.num_people * TIME_TO_RESCUE_PERSON
        if self.target_location in self.survivor_locations:
            self.survivor_locations.remove(self.target_location)
        self.searched_locations.append(self.target_location)
        self.target_location = None

In [ ]:
class simulation():
    """
    Basic governing body for the running of the simulation
    """
    num_timesteps = 24 * 3600
    timesteps_left = num_timesteps
    agent = []
    send_queue = []

    def __init__(self):
        self.generate_locations()
        self.get_locations_with_survivors()
        self.generate_fire_stations()

    def add_agents(self, agents: list[Agent]):
        self.agents = agents

    def tick(self):
        if self.timesteps_left % TIME_BETWEEN_SENDS == 0:
            new_survivor_location = self.get_random_survivor_location
            self.send_queue.append(new_survivor_location)
            closest_agent_index = self.get_closest_agent_to_location(new_survivor_location)
            for i in range(0, len(self.agents)):
                if i == closest_agent_index:
                    self.agents[i].tick(self.locations, closest_agent_index)
                else:
                    self.agents[i].tick(self.locations)
        for agent in self.agents:
            agent.tick(self.locations)

    def generate_locations(self):
        self.locations = []
        num_survivors = TOTAL_POPULATION
        for index, series in louisiana_locations.iterrows():
            x = series['x']
            y = series['y']
            rand_num = np.random.randint(0, 2)
            survivors_at_location = 0
            if rand_num == 1:
                survivors_at_location = np.random.randint(0, MAX_PEOPLE_PER_LOCATION + 1)
            new_location = location(x, y)
            new_location.populate_with_survivors(survivors_at_location)
            self.locations.append(new_location)

    def get_locations_with_survivors(self):
        self.locations_with_survivors = []
        for loc in locations:
            if loc.num_people != 0:
                self.locations_with_survivors.append(loc)

    def generate_fire_stations(self):
        self.stations = []
        for index, series in fire_stations.iterrows():
            x = series['x']
            y = series['y']
            new_station = FireStation(x, y, 1)
            self.stations.append(new_station)

    def get_random_survivor_location(self):
        index = np.random.randint(0, len(self.locations_with_survivors))
        new_survivor_location = self.locations_with_survivors[index]
        self.locations_with_survivors.remove(new_survivor_location)
        return new_survivor_location

    def get_closest_agent_to_location(self, loc: location):
        smallest_distance = np.inf
        closest_agent_index = 0
        for i in range(0, len(self.agents)):
            distance = np.linalg.norm(self.agents[i].current_position - loc)
            if distance < smallest_distance:
                smallest_distance = distance
                closest_agent_index = i
        return closest_agent_index

    def save_state(self, outfile: str):
        pass